In [13]:
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import collections
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC

In [5]:
# read names that have provided survey eye color data
columns = ['name', 'timestamp', 'id', 'blood_type', 'height', 'weight', 'hw_comments', 'left', 'right', 'left_desc', 'right_desc', 'eye_comments', 'hair', 'hair_desc', 'hair_comments', 'misc', 'handedness']

# pgp eye color data from survey
surveyData = pd.read_csv("../eye_color_data/PGP-Survey.csv", names=columns, na_values=['nan', '', 'NaN'])

# names of the pgp participants
surveyNames = np.asarray(surveyData['name'].values.tolist())

# load numpy array of tiled PGP data 
pgp = preprocessing.scale(np.load("../hiq-pgp").astype('double'))
pgp_unscaled = np.load("../hiq-pgp")

In [6]:
# load numpy array of names and keep only the huID
pgpNames = np.load("../names")
pgpNames = map(lambda name: name[:8], pgpNames)

# simple lambda function to return if the input is a string
isstr = lambda val: isinstance(val, str)

In [7]:
eye_color = collections.namedtuple("EyeColor", ['left', 'right'])

# lookup a name in the survey data and return a tuple of the eye colors
def getData(name, surveyData, excludeHazel=False):
    for index, row in surveyData.iterrows():
        if row['name'] == name:
            if not excludeHazel:
                return eye_color(row['left'], row['right'])
            else:
                if isstr(row['left_desc']) and isstr(row['right_desc']):
                    if 'azel' in row['left_desc'] or 'azel' in row['right_desc']:
                        return None
                return eye_color(row['left'], row['right'])

In [8]:
# list of tuples for index and name with eye color data (idx, name)
nameEyeMap = []
namePair = collections.namedtuple("NamePair", ['index', 'name'])

# dictionary of left and right eye colors with respective name, i.e., {"huID": 12}
leftEyeMap = {}
rightEyeMap = {}

existingNames = []

# loop through pgpNames and add eye color to maps, making sure not to add the same name twice
for i, name in enumerate(pgpNames):
    if name in surveyNames and name not in existingNames:
        existingNames.append(name)
        eyeData = getData(name, surveyData, excludeHazel=True)
        if eyeData == None:
            pass
        elif isstr(eyeData.left) and isstr(eyeData.right):
            nameEyeMap.append(namePair(i, name))
            leftEyeMap[name] = eyeData.left
            rightEyeMap[name] = eyeData.right

# create lists containing the known eye color names and the unknown eye colors.
nameIndices, correspondingNames = [], []
for pair in nameEyeMap:
    nameIndices.append(pair.index)
    correspondingNames.append(pair.name)
knownData = pgp[nameIndices]
knownData_unscaled = pgp_unscaled[nameIndices]
unknownData = np.delete(pgp, nameIndices, axis=0)

In [9]:
# convert dictionaries to lists 
leftEyeNameList = []
rightEyeNameList = []
# nametuple looks like (index, name)
for _, name in nameEyeMap:
    if isstr(leftEyeMap[name]):
        leftEyeNameList.append(leftEyeMap[name])
    if isstr(rightEyeMap[name]):
        rightEyeNameList.append(rightEyeMap[name])

blueOrNot = lambda color: 0 if int(color) > 13 else 1
leftEyeNameList = map(blueOrNot, leftEyeNameList)

In [10]:
print knownData.shape

from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
knownData = sel.fit_transform(knownData)
print "after", knownData.shape

(61, 2469062)
after (61, 1146376)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(knownData, leftEyeNameList, test_size=.2, random_state=2)


In [14]:
svc = LinearSVC(penalty='l1', class_weight='balanced', C=.06, max_iter=1000, verbose=True, dual=False)
svc.fit(X_train, y_train)
print accuracy_score(svc.predict(X_test), y_test)

[LibLinear]1.0


In [ ]:
from sklearn.svm import LinearSVC

model = SGDClassifier(penalty='l1', 
                      class_weight='balanced',
                      n_iter=50000, 
                      alpha=0.273,
                      verbose=True, 
                      loss='hinge',
                      warm_start=False,
                      random_state=2,
                      n_jobs=-1,
                      shuffle=True)
                      
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print "Accuracy: ", accuracy_score(y_test, y_pred)

-- Epoch 1
Norm: 2800.13, NNZs: 154284, Bias: -7.024005, T: 48, Avg. loss: 5351.562898
Total training time: 0.69 seconds.
-- Epoch 2
Norm: 2804.27, NNZs: 101772, Bias: -7.323083, T: 96, Avg. loss: 2851.917982
Total training time: 1.16 seconds.
-- Epoch 3
Norm: 2805.61, NNZs: 79924, Bias: -7.477583, T: 144, Avg. loss: 1943.391373
Total training time: 1.60 seconds.
-- Epoch 4
Norm: 2806.01, NNZs: 67879, Bias: -7.536825, T: 192, Avg. loss: 1464.712142
Total training time: 2.02 seconds.
-- Epoch 5
Norm: 2806.24, NNZs: 60146, Bias: -7.579734, T: 240, Avg. loss: 1174.420438
Total training time: 2.41 seconds.
-- Epoch 6
Norm: 2806.50, NNZs: 55420, Bias: -7.637577, T: 288, Avg. loss: 983.239034
Total training time: 2.83 seconds.
-- Epoch 7
Norm: 2806.77, NNZs: 51787, Bias: -7.713922, T: 336, Avg. loss: 846.952329
Total training time: 3.27 seconds.
-- Epoch 8
Norm: 2806.93, NNZs: 48119, Bias: -7.766520, T: 384, Avg. loss: 744.582046
Total training time: 3.68 seconds.
-- Epoch 9
Norm: 2807.08, N

Norm: 2808.22, NNZs: 21614, Bias: -8.863499, T: 3264, Avg. loss: 90.911191
Total training time: 28.27 seconds.
-- Epoch 69
Norm: 2808.22, NNZs: 21558, Bias: -8.869430, T: 3312, Avg. loss: 89.601199
Total training time: 28.65 seconds.
-- Epoch 70
Norm: 2808.23, NNZs: 21443, Bias: -8.878211, T: 3360, Avg. loss: 88.337918
Total training time: 29.04 seconds.
-- Epoch 71
Norm: 2808.23, NNZs: 21352, Bias: -8.886854, T: 3408, Avg. loss: 87.109120
Total training time: 29.43 seconds.
-- Epoch 72
Norm: 2808.23, NNZs: 21237, Bias: -8.893960, T: 3456, Avg. loss: 85.909497
Total training time: 29.82 seconds.
-- Epoch 73
Norm: 2808.24, NNZs: 21147, Bias: -8.902381, T: 3504, Avg. loss: 84.745895
Total training time: 30.21 seconds.
-- Epoch 74
Norm: 2808.24, NNZs: 21084, Bias: -8.909298, T: 3552, Avg. loss: 83.609296
Total training time: 30.60 seconds.
-- Epoch 75
Norm: 2808.24, NNZs: 21008, Bias: -8.917481, T: 3600, Avg. loss: 82.509341
Total training time: 31.00 seconds.
-- Epoch 76
Norm: 2808.24, N

Norm: 2808.32, NNZs: 16696, Bias: -9.255557, T: 6480, Avg. loss: 46.101673
Total training time: 54.37 seconds.
-- Epoch 136
Norm: 2808.32, NNZs: 16456, Bias: -9.262307, T: 6528, Avg. loss: 45.770095
Total training time: 54.79 seconds.
-- Epoch 137
Norm: 2808.33, NNZs: 16421, Bias: -9.265285, T: 6576, Avg. loss: 45.438386
Total training time: 55.15 seconds.
-- Epoch 138
Norm: 2808.33, NNZs: 16388, Bias: -9.271198, T: 6624, Avg. loss: 45.116895
Total training time: 55.56 seconds.
-- Epoch 139
Norm: 2808.33, NNZs: 16360, Bias: -9.275604, T: 6672, Avg. loss: 44.795303
Total training time: 55.94 seconds.
-- Epoch 140
Norm: 2808.33, NNZs: 16340, Bias: -9.280711, T: 6720, Avg. loss: 44.479198
Total training time: 56.34 seconds.
-- Epoch 141
Norm: 2808.33, NNZs: 16325, Bias: -9.285779, T: 6768, Avg. loss: 44.167443
Total training time: 56.73 seconds.
-- Epoch 142
Norm: 2808.33, NNZs: 16306, Bias: -9.290088, T: 6816, Avg. loss: 43.858839
Total training time: 57.12 seconds.
-- Epoch 143
Norm: 28

Norm: 2808.36, NNZs: 14720, Bias: -9.517225, T: 9696, Avg. loss: 30.952246
Total training time: 80.64 seconds.
-- Epoch 203
Norm: 2808.36, NNZs: 14705, Bias: -9.520741, T: 9744, Avg. loss: 30.801527
Total training time: 81.04 seconds.
-- Epoch 204
Norm: 2808.36, NNZs: 14701, Bias: -9.524739, T: 9792, Avg. loss: 30.651593
Total training time: 81.45 seconds.
-- Epoch 205
Norm: 2808.36, NNZs: 14695, Bias: -9.526729, T: 9840, Avg. loss: 30.502724
Total training time: 81.81 seconds.
-- Epoch 206
Norm: 2808.36, NNZs: 14669, Bias: -9.530194, T: 9888, Avg. loss: 30.355981
Total training time: 82.21 seconds.
-- Epoch 207
Norm: 2808.36, NNZs: 14660, Bias: -9.532656, T: 9936, Avg. loss: 30.210358
Total training time: 82.59 seconds.
-- Epoch 208
Norm: 2808.37, NNZs: 14650, Bias: -9.536575, T: 9984, Avg. loss: 30.067511
Total training time: 83.00 seconds.
-- Epoch 209
Norm: 2808.37, NNZs: 14638, Bias: -9.539986, T: 10032, Avg. loss: 29.924487
Total training time: 83.40 seconds.
-- Epoch 210
Norm: 2

Norm: 2808.38, NNZs: 13860, Bias: -9.690991, T: 12864, Avg. loss: 23.381975
Total training time: 107.35 seconds.
-- Epoch 269
Norm: 2808.38, NNZs: 13851, Bias: -9.694022, T: 12912, Avg. loss: 23.295850
Total training time: 107.76 seconds.
-- Epoch 270
Norm: 2808.38, NNZs: 13843, Bias: -9.695155, T: 12960, Avg. loss: 23.209779
Total training time: 108.11 seconds.
-- Epoch 271
Norm: 2808.38, NNZs: 13563, Bias: -9.698539, T: 13008, Avg. loss: 23.125234
Total training time: 108.53 seconds.
-- Epoch 272
Norm: 2808.38, NNZs: 13555, Bias: -9.699662, T: 13056, Avg. loss: 23.040613
Total training time: 108.90 seconds.
-- Epoch 273
Norm: 2808.38, NNZs: 13548, Bias: -9.701901, T: 13104, Avg. loss: 22.957500
Total training time: 109.29 seconds.
-- Epoch 274
Norm: 2808.38, NNZs: 13541, Bias: -9.705991, T: 13152, Avg. loss: 22.874725
Total training time: 109.74 seconds.
-- Epoch 275
Norm: 2808.38, NNZs: 13531, Bias: -9.707473, T: 13200, Avg. loss: 22.791962
Total training time: 110.11 seconds.
-- Ep

...

Norm: 2808.42, NNZs: 8321, Bias: -10.783493, T: 98160, Avg. loss: 3.096067
Total training time: 846.28 seconds.
-- Epoch 2046
Norm: 2808.42, NNZs: 8321, Bias: -10.783842, T: 98208, Avg. loss: 3.094561
Total training time: 846.71 seconds.
-- Epoch 2047
Norm: 2808.42, NNZs: 8320, Bias: -10.783891, T: 98256, Avg. loss: 3.093050
Total training time: 847.06 seconds.
-- Epoch 2048
Norm: 2808.42, NNZs: 8320, Bias: -10.784289, T: 98304, Avg. loss: 3.091549
Total training time: 847.49 seconds.
-- Epoch 2049
Norm: 2808.42, NNZs: 8319, Bias: -10.784487, T: 98352, Avg. loss: 3.090045
Total training time: 847.95 seconds.
-- Epoch 2050
Norm: 2808.42, NNZs: 8319, Bias: -10.784785, T: 98400, Avg. loss: 3.088541
Total training time: 848.34 seconds.
-- Epoch 2051
Norm: 2808.42, NNZs: 8319, Bias: -10.784934, T: 98448, Avg. loss: 3.087038
Total training time: 848.70 seconds.
-- Epoch 2052
Norm: 2808.42, NNZs: 8318, Bias: -10.785182, T: 98496, Avg. loss: 3.085537
Total training time: 849.08 seconds.
-- Epo

Norm: 2808.42, NNZs: 8270, Bias: -10.799020, T: 101280, Avg. loss: 3.000974
Total training time: 873.09 seconds.
-- Epoch 2111
Norm: 2808.42, NNZs: 8269, Bias: -10.799261, T: 101328, Avg. loss: 2.999556
Total training time: 873.46 seconds.
-- Epoch 2112
Norm: 2808.42, NNZs: 8267, Bias: -10.799453, T: 101376, Avg. loss: 2.998141
Total training time: 873.83 seconds.
-- Epoch 2113
Norm: 2808.42, NNZs: 8266, Bias: -10.799694, T: 101424, Avg. loss: 2.996726
Total training time: 874.20 seconds.
-- Epoch 2114
Norm: 2808.42, NNZs: 8265, Bias: -10.799935, T: 101472, Avg. loss: 2.995316
Total training time: 874.57 seconds.
-- Epoch 2115
Norm: 2808.42, NNZs: 8265, Bias: -10.800127, T: 101520, Avg. loss: 2.993905
Total training time: 874.93 seconds.
-- Epoch 2116
Norm: 2808.42, NNZs: 8264, Bias: -10.800416, T: 101568, Avg. loss: 2.992497
Total training time: 875.31 seconds.
-- Epoch 2117
Norm: 2808.42, NNZs: 8264, Bias: -10.800560, T: 101616, Avg. loss: 2.991087
Total training time: 875.68 seconds

Norm: 2808.42, NNZs: 8235, Bias: -10.814121, T: 104400, Avg. loss: 2.911563
Total training time: 900.32 seconds.
-- Epoch 2176
Norm: 2808.42, NNZs: 8235, Bias: -10.814261, T: 104448, Avg. loss: 2.910226
Total training time: 900.67 seconds.
-- Epoch 2177
Norm: 2808.42, NNZs: 8234, Bias: -10.814495, T: 104496, Avg. loss: 2.908892
Total training time: 901.04 seconds.
-- Epoch 2178
Norm: 2808.42, NNZs: 8232, Bias: -10.814681, T: 104544, Avg. loss: 2.907560
Total training time: 901.40 seconds.
-- Epoch 2179
Norm: 2808.42, NNZs: 8227, Bias: -10.814915, T: 104592, Avg. loss: 2.906229
Total training time: 901.77 seconds.
-- Epoch 2180
Norm: 2808.42, NNZs: 8227, Bias: -10.815148, T: 104640, Avg. loss: 2.904900
Total training time: 902.14 seconds.
-- Epoch 2181
Norm: 2808.42, NNZs: 8227, Bias: -10.815335, T: 104688, Avg. loss: 2.903571
Total training time: 902.50 seconds.
-- Epoch 2182
Norm: 2808.42, NNZs: 8226, Bias: -10.815615, T: 104736, Avg. loss: 2.902247
Total training time: 902.93 seconds

Norm: 2808.42, NNZs: 8186, Bias: -10.828317, T: 107520, Avg. loss: 2.827302
Total training time: 926.27 seconds.
-- Epoch 2241
Norm: 2808.42, NNZs: 8185, Bias: -10.828635, T: 107568, Avg. loss: 2.826042
Total training time: 926.70 seconds.
-- Epoch 2242
Norm: 2808.42, NNZs: 8185, Bias: -10.828771, T: 107616, Avg. loss: 2.824784
Total training time: 927.07 seconds.
-- Epoch 2243
Norm: 2808.42, NNZs: 8182, Bias: -10.829043, T: 107664, Avg. loss: 2.823528
Total training time: 927.46 seconds.
-- Epoch 2244
Norm: 2808.42, NNZs: 8182, Bias: -10.829179, T: 107712, Avg. loss: 2.822274
Total training time: 927.83 seconds.
-- Epoch 2245
Norm: 2808.42, NNZs: 8182, Bias: -10.829406, T: 107760, Avg. loss: 2.821020
Total training time: 928.21 seconds.
-- Epoch 2246
Norm: 2808.42, NNZs: 8182, Bias: -10.829587, T: 107808, Avg. loss: 2.819768
Total training time: 928.60 seconds.
-- Epoch 2247
Norm: 2808.42, NNZs: 8181, Bias: -10.829859, T: 107856, Avg. loss: 2.818518
Total training time: 929.01 seconds

Norm: 2808.42, NNZs: 8148, Bias: -10.842152, T: 110640, Avg. loss: 2.747778
Total training time: 952.71 seconds.
-- Epoch 2306
Norm: 2808.42, NNZs: 8147, Bias: -10.842285, T: 110688, Avg. loss: 2.746587
Total training time: 953.08 seconds.
-- Epoch 2307
Norm: 2808.42, NNZs: 8147, Bias: -10.842549, T: 110736, Avg. loss: 2.745401
Total training time: 953.51 seconds.
-- Epoch 2308
Norm: 2808.42, NNZs: 8147, Bias: -10.842638, T: 110784, Avg. loss: 2.744214
Total training time: 953.86 seconds.
-- Epoch 2309
Norm: 2808.42, NNZs: 8147, Bias: -10.842902, T: 110832, Avg. loss: 2.743030
Total training time: 954.24 seconds.
-- Epoch 2310
Norm: 2808.42, NNZs: 8147, Bias: -10.843078, T: 110880, Avg. loss: 2.741846
Total training time: 954.61 seconds.
-- Epoch 2311
Norm: 2808.42, NNZs: 8147, Bias: -10.843343, T: 110928, Avg. loss: 2.740663
Total training time: 955.00 seconds.
-- Epoch 2312
Norm: 2808.42, NNZs: 8147, Bias: -10.843519, T: 110976, Avg. loss: 2.739482
Total training time: 955.36 seconds

In [ ]:
randomArray = np.random.randint(10, size=(500, 2469062))
results = []

for index, arr in enumerate(randomArray):
    if arr[1792420] % 2 == 0:
        randomArray[index][1792420] = 0
        results.append(0)
    else:
        randomArray[index][1792420] = 1
        results.append(1)

In [29]:
print cv_scores.mean(), cv_scores.std()

0.54380952381 0.0860785908895


In [29]:
nonzeroes = np.nonzero(model.coef_)[1]
coefs = zip(nonzeroes, abs(model.coef_[0][nonzeroes]))
coefs.sort(key = lambda x: x[1], reverse=True)

In [30]:
coefs

[(2386364, 7.441622731249633),
 (2337764, 7.3935125486476787),
 (2337765, 7.3935125486476787),
 (1937403, 7.124263254730705),
 (946501, 6.8653090147339304),
 (621409, 6.6819260362721611),
 (1776401, 6.6795449024968736),
 (603153, 6.6504918319263728),
 (170718, 6.6230700349812075),
 (170724, 6.6230700349812075),
 (923854, 6.6230700349812075),
 (1069426, 6.6230700349812075),
 (1177137, 6.5325547301549474),
 (362283, 6.428266974002212),
 (835769, 6.3188942811264583),
 (1403333, 6.2795478513664564),
 (1822747, 6.2525774976035251),
 (22919, 6.1895035575049855),
 (30489, 6.1895035575049855),
 (37541, 6.1895035575049855),
 (66113, 6.1895035575049855),
 (68271, 6.1895035575049855),
 (68721, 6.1895035575049855),
 (72843, 6.1895035575049855),
 (79373, 6.1895035575049855),
 (85001, 6.1895035575049855),
 (119811, 6.1895035575049855),
 (127437, 6.1895035575049855),
 (199155, 6.1895035575049855),
 (205427, 6.1895035575049855),
 (225079, 6.1895035575049855),
 (284501, 6.1895035575049855),
 (298035, 6

In [ ]:
arange = np.logspace(-2, 4, 10).tolist()
means = []
stds = []
for Aval in arange:
    svc_test = SGDClassifier(penalty='l1', class_weight='balanced', alpha=Aval, n_iter=1000, shuffle=True)
    cv_score = cross_val_score(svc_test, knownData, leftEyeNameList, cv=10, scoring='accuracy')
    means.append(cv_score.mean())
    stds.append(cv_score.std())
    print "alpha:", Aval, "mean:", cv_score.mean(), "std:", cv_score.std()